# Analysis of Moment Estimation Techniques

In this notebook we compare the performance of different moment esimtation techniques with output from simuPOP. The data used in this notebook can be generated using the snakefile in the folder `paperImages/fixedImages`.

We first specificy the column names and read in the data. All data is stored in `odePerformance_all.csv`.

In [1]:
# Specify column names and load data
import pandas as pd

column_names = ['interp_type', 'initial_value', 'demography', 'renomalize_ind', 'parsimonious_ind', 'min_step', 's', 'r',
               'bot_abs_error', 'exp_abs_error', 'tot_abs_error', 
                'bot_abs_weighted_error', 'exp_abs_weighted_error', 'tot_abs_weighted_error',
                'bot_mse', 'exp_mse', 'tot_mse']
performance = pd.read_csv('odePerformance_all.csv', names=column_names)
performance['interp_type'].replace('jackknife', 'jackknife-unconstrained', inplace=True)
performance['interp_type'].replace('jackknife-constrained', 'jackknife', inplace=True)
performance.head()

,interp_type,initial_value,demography,renomalize_ind,parsimonious_ind,min_step,s,r,bot_abs_error,exp_abs_error,tot_abs_error,bot_abs_weighted_error,exp_abs_weighted_error,tot_abs_weighted_error,bot_mse,exp_mse,tot_mse
0,jackknife,"[0.01, 0.0, 0.49, 0.5]",bottle,False,True,0.0001,0.00025,0.01,9.451778e-19,0.000019,0.000014,NaN,13.934344,1029.622446,4.219092e-34,6.285817e-08,2.534171e-08
1,jackknife,"[0.01, 0.0, 0.49, 0.5]",bottle,False,False,0.0001,0.00025,0.01,9.451778e-19,0.000110,0.000234,NaN,51.070376,454999.992871,4.219092e-34,3.005296e-06,1.790325e-05
2,jackknife,"[0.01, 0.0, 0.49, 0.5]",bottle,True,True,0.0001,0.00025,0.01,6.582540e-19,0.000019,0.000014,NaN,13.942102,1030.606083,1.574846e-34,6.290040e-08,2.527101e-08
3,jackknife,"[0.01, 0.0, 0.49, 0.5]",bottle,True,False,0.0001,0.00025,0.01,6.582540e-19,0.000074,0.000137,NaN,30.244547,157542.883412,1.574846e-34,1.238216e-06,5.962622e-06
4,jackknife-unconstrained,"[0.01, 0.0, 0.49, 0.5]",bottle,False,True,0.0001,0.00025,0.01,9.451778e-19,0.000019,0.000014,NaN,13.933481,1044.375954,4.219092e-34,6.287751e-08,2.532837e-08


In some cases, the approximations performed so poorly that the ODE integration needed to be aborted. In these cases we replace the resulting `NA` with 10 times the maximum error.

In [2]:
# Fill missing values with 10 times the worst performance
performance.fillna(performance.max()*10, inplace=True)
performance.head()

,interp_type,initial_value,demography,renomalize_ind,parsimonious_ind,min_step,s,r,bot_abs_error,exp_abs_error,tot_abs_error,bot_abs_weighted_error,exp_abs_weighted_error,tot_abs_weighted_error,bot_mse,exp_mse,tot_mse
0,jackknife,"[0.01, 0.0, 0.49, 0.5]",bottle,False,True,0.0001,0.00025,0.01,9.451778e-19,0.000019,0.000014,inf,13.934344,1029.622446,4.219092e-34,6.285817e-08,2.534171e-08
1,jackknife,"[0.01, 0.0, 0.49, 0.5]",bottle,False,False,0.0001,0.00025,0.01,9.451778e-19,0.000110,0.000234,inf,51.070376,454999.992871,4.219092e-34,3.005296e-06,1.790325e-05
2,jackknife,"[0.01, 0.0, 0.49, 0.5]",bottle,True,True,0.0001,0.00025,0.01,6.582540e-19,0.000019,0.000014,inf,13.942102,1030.606083,1.574846e-34,6.290040e-08,2.527101e-08
3,jackknife,"[0.01, 0.0, 0.49, 0.5]",bottle,True,False,0.0001,0.00025,0.01,6.582540e-19,0.000074,0.000137,inf,30.244547,157542.883412,1.574846e-34,1.238216e-06,5.962622e-06
4,jackknife-unconstrained,"[0.01, 0.0, 0.49, 0.5]",bottle,False,True,0.0001,0.00025,0.01,9.451778e-19,0.000019,0.000014,inf,13.933481,1044.375954,4.219092e-34,6.287751e-08,2.532837e-08


For comparison we will only focus on iterations under the full demography (i.e. 6,000 generations at population size 10,000, followed by 3,000 generations at 2,000, followed by 1,000 generations of exponential growth).

In [3]:
# Include only when cases with with the full demography
performance = performance[performance['demography']=='full']

We now count all of the entries to check that we have 48 cases for each combination of interpolation type.

In [4]:
# Make sure there are 48 simulations for each ode setting
to_group = ['interp_type', 'renomalize_ind', 'parsimonious_ind']
performance_counts = performance.groupby(to_group).count()
performance_counts

initial_value  \
interp_type             renomalize_ind parsimonious_ind                  
jackknife               False          False                        48   
                                       True                         48   
                        True           False                        48   
                                       True                         48   
jackknife-unconstrained False          False                        48   
                                       True                         48   
                        True           False                        48   
                                       True                         48   
lin                     False          False                        48   
                                       True                         48   
                        True           False                        48   
                                       True                         48   
loglin                  False          False                        48   
                                       True                         48   
                        True           False                        48   
                                       True                         48   
project                 False          False                        48   
                                       True                         48   
                        True           False                        48   
                                       True                         48   

                                                         demography  min_step  \
interp_type             renomalize_ind parsimonious_ind                         
jackknife               False          False                     48        48   
                                       True                      48        48   
                        True           False                     48        48   
                                       True                      48        48   
jackknife-unconstrained False          False                     48        48   
                                       True                      48        48   
                        True           False                     48        48   
                                       True                      48        48   
lin                     False          False                     48        48   
                                       True                      48        48   
                        True           False                     48        48   
                                       True                      48        48   
loglin                  False          False                     48        48   
                                       True                      48        48   
                        True           False                     48        48   
                                       True                      48        48   
project                 False          False                     48        48   
                                       True                      48        48   
                        True           False                     48        48   
                                       True                      48        48   

                                                          s   r  \
interp_type             renomalize_ind parsimonious_ind           
jackknife               False          False             48  48   
                                       True              48  48   
                        True           False             48  48   
                                       True              48  48   
jackknife-unconstrained False          False             48  48   
                                       True              48  48   
                        True           False             48  48   
                                       Tru

Below we take the mean over all interations for all interpolation options and find that `loglin` with renormalization but no parsimonious projection has the best performance.

In [5]:
# Select metrics and parameters to group by
metrics = ['tot_abs_error', 'tot_mse']


# Compute means for each configuration
performance_means = performance.groupby(to_group)[metrics].mean()

# Get best performer for each metric
min_mean_idx = performance_means.idxmin()

# Print best performers and means
print(min_mean_idx)
performance_means.loc[min_mean_idx]

tot_abs_error    (loglin, True, False)
tot_mse          (loglin, True, False)
dtype: object


tot_abs_error       tot_mse
interp_type renomalize_ind parsimonious_ind                             
loglin      True           False                  0.000015  7.443173e-08
                           False                  0.000015  7.443173e-08

And below we get the best combination of renomalization and parsimonious projection for each interpolation type (according to MSE) along with the corresponding MSE.

In [6]:
# Select best MSE configuration for each interpolation type
performance_means.loc[performance_means.groupby('interp_type').idxmin()['tot_mse']]

,,,tot_abs_error,tot_mse
interp_type,renomalize_ind,parsimonious_ind,,
jackknife,True,True,0.000049,1.055580e-05
jackknife-unconstrained,True,True,0.000043,8.701658e-06
lin,True,False,0.000044,1.070619e-06
loglin,True,False,0.000015,7.443173e-08
project,True,False,0.000018,1.009133e-06


We then do the same for absolute error. Notice that the only change is in the `lin` interpolation type.

In [7]:
# Select best Absolute error configuration for eahc interpolation type
performance_means.loc[performance_means.groupby('interp_type').idxmin()['tot_abs_error']]




,,,tot_abs_error,tot_mse
interp_type,renomalize_ind,parsimonious_ind,,
jackknife,True,True,0.000049,1.055580e-05
jackknife-unconstrained,True,True,0.000043,8.701658e-06
lin,True,True,0.000022,2.944640e-06
loglin,True,False,0.000015,7.443173e-08
project,True,False,0.000018,1.009133e-06


We now perform the same analysis by considering the mini-max performance. Namely, we find the combination of interpolation parameters which has the best performance when considering its worst interation. Below we see that `loglin` along with renormalization but not projection performs best again by this metric.

In [8]:
# Compute worst trail for each configuration
performance_maxs_idx = performance.groupby(to_group)[metrics].idxmax()
performance_maxs = performance.groupby(to_group)[metrics].max()

# Print configuration which has the minimax performance
max_min_idxs = performance_maxs.idxmin()
print(max_min_idxs)
performance_maxs.loc[max_min_idxs]

tot_abs_error    (loglin, True, False)
tot_mse          (loglin, True, False)
dtype: object


tot_abs_error       tot_mse
interp_type renomalize_ind parsimonious_ind                             
loglin      True           False                  0.000024  8.439334e-07
                           False                  0.000024  8.439334e-07

Then we display the mini-max MSE for each interpolation type.

In [9]:
# Select best MSE configuration for each interpolation type
performance_maxs.loc[performance_maxs.groupby('interp_type').idxmin()['tot_mse']]

,,,tot_abs_error,tot_mse
interp_type,renomalize_ind,parsimonious_ind,,
jackknife,True,True,0.000254,1.073714e-04
jackknife-unconstrained,True,False,0.000334,2.840697e-05
lin,True,False,0.000120,1.084015e-05
loglin,True,False,0.000024,8.439334e-07
project,True,False,0.000159,4.300009e-05


We do the same for absolute error. In this case `jackknife-unconstrained` is the only interpolation type with different combinations of paramters depending on the error metric.

In [10]:
# Select best Absolute error configuration for eahc interpolation type
performance_maxs.loc[performance_maxs.groupby('interp_type').idxmin()['tot_abs_error']]

,,,tot_abs_error,tot_mse
interp_type,renomalize_ind,parsimonious_ind,,
jackknife,True,True,0.000254,1.073714e-04
jackknife-unconstrained,True,True,0.000240,1.012991e-04
lin,True,False,0.000120,1.084015e-05
loglin,True,False,0.000024,8.439334e-07
project,True,False,0.000159,4.300009e-05
